In [2]:
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('C:/Users/dell/Downloads/vehicle price prediction/final step data.csv')
df=df.drop(columns=['mmr','seller'])

m_df=df

In [3]:
#df['condition']=df['condition']/10
print(df.shape)

print(df.info())
df.sample(2)

(548535, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548535 entries, 0 to 548534
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          548535 non-null  int64  
 1   make          548535 non-null  object 
 2   model         548535 non-null  object 
 3   body          548535 non-null  object 
 4   condition     548535 non-null  float64
 5   odometer      548535 non-null  float64
 6   sellingprice  548535 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 29.3+ MB
None


,year,make,model,body,condition,odometer,sellingprice
89505,2011,toyota,camry,sedan,49.0,16992.0,12300.0
57044,2008,dodge,charger,sedan,23.0,141004.0,4500.0


In [4]:
df.describe()
df=df.dropna()

In [5]:
target_transformer = PowerTransformer(method='yeo-johnson')
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVR


X = df.drop('sellingprice', axis=1)
y = df['sellingprice']
y_transformed =FunctionTransformer(np.log1p).fit_transform(y) #target_transformer.fit_transform(y.values.reshape(-1, 1)).ravel()  # flatten

numerical_features = ['condition', 'odometer']     # Numerical columns
categorical_features = ['make', 'model', 'body']  # Categorical columns
year_col = ['year']  # Year is numerical but treated separately


#inp_trns=PowerTransformer(method='yeo-johnson')

In [6]:

# Step 4: Define Preprocessor
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = Pipeline(steps=[

    ('log1p', FunctionTransformer(np.log1p)),

   # ('power',PowerTransformer(method='yeo-johnson')) ,
    ('scaler', StandardScaler())
])


year_transformer = Pipeline(steps=[
    ('log1p', FunctionTransformer(np.log1p)),
    #('power', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features),
    ('year', year_transformer, year_col)
])





In [7]:
from sklearn.decomposition import PCA

# Step 5: Define Pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('pca',PCA(n_components=70) ),

    ('regressor',XGBRegressor())
])

In [8]:
scores = cross_val_score(pipeline, X, y_transformed, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
print("Cross-Validated RMSE:", rmse_scores)
print("Average RMSE:", np.mean(rmse_scores))


Cross-Validated RMSE: [0.30273901 0.24010318 0.23670183 0.24468589 0.24281527]
Average RMSE: 0.25340903411964383


In [ ]:
from sklearn.model_selection import GridSearchCV
# Step 9: Fit full pipeline
pipeline.fit(X, y_transformed)
param_grid = {
    'regressor__n_estimators': [200, 300],        # कितने trees (ज्यादा value = बेहतर learning but slow)
    'regressor__max_depth': [ 5, 7],                 # हर tree की गहराई (deep = ज्यादा complex)
    'regressor__learning_rate': [ 0.05, 0.1],     # छोटा learning rate → stable but slow learning
    'regressor__subsample': [0.8, 1.0],                # हर tree के लिए कितने data points use हों
    'regressor__colsample_bytree': [0.7, 1.0],         # हर tree के लिए कितने features use हों
    'regressor__gamma': [0, 1],                        # Regularization (0 = कोई रोक नहीं)
    'regressor__reg_lambda': [0, 1],                   # L2 regularization
}
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=2)
grid.fit(X, y_transformed)

print("Best R²:", grid.best_score_)
print("Best Parameters:", grid.best_params_)
# Optional: Evaluate
y_pred = cross_val_predict(pipeline, X, y_transformed, cv=5)
print("\nEvaluation Metrics:")
print("RMSE:", np.sqrt(mean_squared_error(y_transformed, y_pred)))
print("MAE:", mean_absolute_error(y_transformed, y_pred))
print("R2 Score:", r2_score(y_transformed, y_pred))

Fitting 5 folds for each of 128 candidates, totalling 640 fits


In [ ]:
# # Step 10: Save pipeline & target transformer
# import pickle
# with open("xgboost3_model_pipeline.pkl", "wb") as file:
#     pickle.dump(pipeline, file)
# with open("inp_transformer.pkl", "wb") as f:
#     pickle.dump(inp_trns, f)

# with open("target_transformer.pkl", "wb") as f:
#     pickle.dump(target_transformer, f)

# Optional: Evaluate